In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-201086
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-201086


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacitycluster1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [19]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": choice(0.1, 1, 5, 10),
    "max_iter": choice(500, 1000, 4000, 10000)
}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1, delay_evaluation=5 )

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=cpu_cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=ps, policy=policy, primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4)

In [20]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "trainrun1")
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)
RunDetails(run).show()

RunId: HD_5ddcc9ed-99f8-49c3-a855-01204d794677
Web View: https://ml.azure.com/runs/HD_5ddcc9ed-99f8-49c3-a855-01204d794677?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-201086/workspaces/quick-starts-ws-201086&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-07-15T03:31:55.633173][API][INFO]Experiment created
[2022-07-15T03:31:56.258094][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-07-15T03:31:57.2274268Z][SCHEDULER][INFO]Scheduling job, id='HD_5ddcc9ed-99f8-49c3-a855-01204d794677_0' 
[2022-07-15T03:31:57.3255125Z][SCHEDULER][INFO]Scheduling job, id='HD_5ddcc9ed-99f8-49c3-a855-01204d794677_1' 
[2022-07-15T03:31:57.4192771Z][SCHEDULER][INFO]Scheduling job, id='HD_5ddcc9ed-99f8-49c3-a855-01204d794677_2' 
[2022-07-15T03:31:57.498081][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-07-15T03:31:57.5590488Z][SCHEDULER][INFO

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [21]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

{'runId': 'HD_5ddcc9ed-99f8-49c3-a855-01204d794677_2',
 'target': 'udacitycluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-07-15T03:38:01.921626Z',
 'endTimeUtc': '2022-07-15T03:40:05.729038Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '9f132b02-6147-4e41-877f-447176ae35a5',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '10', '--max_iter', '4000'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'udacitycluster1',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': 

In [46]:
src_bestrun = ScriptRunConfig(source_directory=".", script="train.py", compute_target=cpu_cluster, environment=sklearn_env,arguments=['--C', 10, '--max_iter',4000])
experiment = Experiment(ws, "bestrun1")
run2 = experiment.submit(config=src_bestrun)
run2.wait_for_completion(show_output=True)

RunId: testrun3_1657865594_bced7abb
Web View: https://ml.azure.com/runs/testrun3_1657865594_bced7abb?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-201086/workspaces/quick-starts-ws-201086&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: testrun3_1657865594_bced7abb
Web View: https://ml.azure.com/runs/testrun3_1657865594_bced7abb?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-201086/workspaces/quick-starts-ws-201086&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'testrun3_1657865594_bced7abb',
 'target': 'udacitycluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-07-15T06:16:07.374612Z',
 'endTimeUtc': '2022-07-15T06:17:27.873683Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': 'a4abb2a4-29db-426b-90e6-39d9eb8c4068',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '10', '--max_iter', '4000'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'udacitycluster1',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'envi

In [28]:
run2.download_file(name="model.pkl")

In [29]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds =TabularDatasetFactory.from_delimited_files(path=web_path)

In [39]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = pd.concat([x,y],axis=1)

In [40]:
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0
5,58,1,0,1,1,7,5,605,1,999,...,0,0,0,0,0,0,1,0,0,0
6,48,1,0,1,0,5,3,243,1,999,...,1,0,0,0,1,0,0,0,0,0
7,51,0,0,1,0,8,4,24,7,999,...,0,0,0,0,0,0,0,1,0,0
8,24,1,0,1,1,6,3,126,4,999,...,1,0,0,0,0,0,0,1,0,0
9,36,0,0,1,1,7,1,43,4,999,...,0,0,0,0,0,0,1,0,0,0


In [42]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y',
    n_cross_validations=5)

In [43]:
# Submit your automl run
experiment = Experiment(ws, "automlrun1")
run3 = experiment.submit(config=automl_config)
run3.wait_for_completion(show_output=True)

2022-07-15:05:20:04,221 WARNING  [local_experiment_driver.py:145] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automlrun1,AutoML_c97aa8e5-6f9b-48a7-927b-8bafaf96915d,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-07-15:05:34:37,952 WARNING  [connectionpool.py:810] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /artifact/v2.0/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourceGroups/aml-quickstarts-201086/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-201086/artifacts/prefix/contentinfo/ExperimentRun/dcid.HD_5ddcc9ed-99f8-49c3-a855-01204d794677/user_logs
2022-07-15:05:53:24,514 INFO     [explanation_client.py:334] Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
automlrun1,AutoML_c97aa8e5-6f9b-48a7-927b-8bafaf96915d,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

2022-07-15:05:53:35,670 WARNING  [connectionpool.py:810] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /metric/v2.0/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourceGroups/aml-quickstarts-201086/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-201086/runs/AutoML_c97aa8e5-6f9b-48a7-927b-8bafaf96915d_0/full


{'runId': 'AutoML_c97aa8e5-6f9b-48a7-927b-8bafaf96915d',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-07-15T05:20:05.977506Z',
 'endTimeUtc': '2022-07-15T05:52:05.957787Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automlrun1","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-201086","workspace_name":"quick-starts-ws-201086","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enab

In [44]:
# Retrieve and save your best automl model.
best_run, fitted_model = run3.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automlrun1,
Id: AutoML_c97aa8e5-6f9b-48a7-927b-8bafaf96915d_17,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_sta

In [45]:
joblib.dump(value=fitted_model,filename='automlmodel.pkl')

['automlmodel.pkl']